<a href="https://colab.research.google.com/github/sukilsiva/Machine-Learnig-Deep-Learning-Code/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
%matplotlib inline

In [3]:
!git clone https://github.com/arvindrvs/nsfw_face.git

fatal: destination path 'nsfw_face' already exists and is not an empty directory.


In [0]:
image_dr = glob("/content/nsfw_face/dataSet/*")

crop = (30,55,155, 170)
images = [np.array((Image.open(i).crop(crop)).resize((64,64))) for i in image_dr]

for i in range(len(images)):
    images[i] = ((images[i] - images[i].min())/(255 - images[i].min()))
    images[i] = images[i]*2-1
    
images = np.array(images)

In [5]:
images

array([[[ 0.41176471,  0.40392157,  0.35686275, ..., -1.        ,
         -1.        , -1.        ],
        [-0.6627451 , -0.6627451 , -0.6627451 , ..., -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        ...,
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ]],

       [[ 0.27027027,  0.27027027,  0.24324324, ...,  0.25225225,
          0.26126126,  0.27027027],
        [ 0.27927928,  0.25225225,  0.23423423, ...,  0.27027027,
          0.26126126,  0.26126126],
        [ 0.27027027,  0.24324324,  0.23423423, ...,  0.28828829,
          0.27027027,  0.26126126],
        ...,
        [ 0.22522523,  0.18918919,  0.1981982 , ..., -

## **THE GENERATOR**

In [0]:
def generator(z,reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        hidden1 = tf.layers.dense(inputs=z,units=128)
        # Leaky Relu
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1,hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1,units=128)
        
        hidden2 = tf.maximum(alpha*hidden2,hidden2)
        output = tf.layers.dense(hidden2,units=784,activation=tf.nn.tanh)
        return output

## **THE DISCRIMINATOR**

In [0]:
def discriminator(x, reuse = None):
  with tf.variable_scope("dis", reuse = reuse):
    hidden1=tf.layers.dense(inputs=x,units =128)
    #here im going to use leaky relu as activation function
    alpha=0.01
    hidden1=tf.maximum(alpha*hidden1,hidden1)
    hidden2=tf.layers.dense(inputs=hidden1,units=128)
    hidden2=tf.maximum(alpha*hidden2,hidden2)
    logits=tf.layers.dense(hidden2,units=1)
    output=tf.sigmoid(logits)
    return logits, output

# **THE PLACEHOLDERS**

In [0]:
z =tf.placeholder(tf.float32, shape = [None,100])
real_images =tf.placeholder (tf.float32, shape = [None,784])

In [9]:
real_images

<tf.Tensor 'Placeholder_1:0' shape=(?, 784) dtype=float32>

In [10]:
z

<tf.Tensor 'Placeholder:0' shape=(?, 100) dtype=float32>

## **GENERATOR**

In [11]:
G = generator(z)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


# **DISCIMINATOR**

In [0]:
D_output_real , D_logits_real = discriminator(real_images)
D_output_fake, D_logits_fake = discriminator(G, reuse=True)

## **LOSSES**

In [0]:
def loss_func(logits_in, labels_in):
  return tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in)

In [14]:
D_real_loss=loss_func(D_logits_real,tf.ones_like(D_logits_real)*(0.9))
D_fake_loss=loss_func(D_logits_fake,tf.zeros_like(D_logits_real))
d_loss = D_real_loss + D_fake_loss

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
g_loss = loss_func(D_logits_fake,tf.zeros_like(D_logits_fake))

# **LEARNING RATE**

In [0]:
learning_rate = 0.001

In [17]:
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

['dis/dense/kernel:0', 'dis/dense/bias:0', 'dis/dense_1/kernel:0', 'dis/dense_1/bias:0', 'dis/dense_2/kernel:0', 'dis/dense_2/bias:0']
['gen/dense/kernel:0', 'gen/dense/bias:0', 'gen/dense_1/kernel:0', 'gen/dense_1/bias:0', 'gen/dense_2/kernel:0', 'gen/dense_2/bias:0']


In [0]:
D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

## **TRAINIG SESSION**

In [0]:
batch_size = 100
epochs = 50
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=g_vars)

In [0]:
samples = []

In [26]:
with tf.Session() as sess:
    
    sess.run(init)
    
    # Recall an epoch is an entire run through the training data
    for e in range(epochs):
        # // indicates classic division
        num_batches = len(images) // batch_size
        
        for i in range(num_batches):
            
            # Grab batch of images
            batch = image_dr(batch_size)
            
            # Get images, reshape and rescale to pass to D
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2 - 1
            
            # Z (random latent noise data for Generator)
            # -1 to 1 because of tanh activation
            batch_z = np.random.uniform(-1, 1, size=(batch_size, 100))
            
            # Run optimizers, no need to save outputs, we won't use them
            _ = sess.run(D_trainer, feed_dict={real_images: batch_images, z: batch_z})
            _ = sess.run(G_trainer, feed_dict={z: batch_z})
        
            
        print("Currently on Epoch {} of {} total...".format(e+1, epochs))
        
        # Sample from generator as we're training for viewing afterwards
        sample_z = np.random.uniform(-1, 1, size=(1, 100))
        gen_sample = sess.run(generator(z ,reuse=True),feed_dict={z: sample_z})
        
        samples.append(gen_sample)
        
#         saver.save(sess, './models/500_epoch_model.ckpt')


Currently on Epoch 1 of 50 total...
Currently on Epoch 2 of 50 total...
Currently on Epoch 3 of 50 total...
Currently on Epoch 4 of 50 total...
Currently on Epoch 5 of 50 total...
Currently on Epoch 6 of 50 total...
Currently on Epoch 7 of 50 total...
Currently on Epoch 8 of 50 total...
Currently on Epoch 9 of 50 total...
Currently on Epoch 10 of 50 total...
Currently on Epoch 11 of 50 total...
Currently on Epoch 12 of 50 total...
Currently on Epoch 13 of 50 total...
Currently on Epoch 14 of 50 total...
Currently on Epoch 15 of 50 total...
Currently on Epoch 16 of 50 total...
Currently on Epoch 17 of 50 total...
Currently on Epoch 18 of 50 total...
Currently on Epoch 19 of 50 total...
Currently on Epoch 20 of 50 total...
Currently on Epoch 21 of 50 total...
Currently on Epoch 22 of 50 total...
Currently on Epoch 23 of 50 total...
Currently on Epoch 24 of 50 total...
Currently on Epoch 25 of 50 total...
Currently on Epoch 26 of 50 total...
Currently on Epoch 27 of 50 total...
Currently 